In [ ]:
import threading
import socket
import time
import cv2
import numpy as np
from collections import Counter
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image

face_classifier = cv2.CascadeClassifier(r'C://emotion//haarcascade_frontalface_default (1).xml')
classifier = load_model(r'C://emotion//model.h5')
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
emotions_list = []  # List to keep track of detected emotions

latest_emotion = "Neutral"  # Default emotion

def calculate_mode(emotions):
    filtered_emotions = [emotion for emotion in emotions if emotion != 'Neutral']
    if filtered_emotions:
        emotion_counts = Counter(filtered_emotions)
        most_common_emotion = emotion_counts.most_common(1)[0][0]
        return most_common_emotion
    return None

def detect_emotions():
    global latest_emotion
    global emotions_list  # Declare the global variable

    cap = cv2.VideoCapture(0)

    while True:
        _, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray)

        detected_emotions = []

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)
            roi_gray = gray[y:y + h, x:x + w]
            roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

            if np.sum([roi_gray]) != 0:
                roi = roi_gray.astype('float') / 255.0
                roi = img_to_array(roi)
                roi = np.expand_dims(roi, axis=0)

                prediction = classifier.predict(roi)[0]
                label = emotion_labels[prediction.argmax()]
                detected_emotions.append(label)
            else:
                cv2.putText(frame, 'No Faces', (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Update the global emotions list and calculate mode
        emotions_list.extend(detected_emotions)
        latest_emotion = calculate_mode(emotions_list)
        emotions_list = []  # Clear list to avoid overfilling

        if latest_emotion:
            cv2.putText(frame, f'Most Frequent Emotion: {latest_emotion}', (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow('Emotion Detector', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def handle_client(conn):
    global latest_emotion
    while True:
        try:
            if latest_emotion:
                conn.sendall(latest_emotion.encode())
            time.sleep(1)  # Update every second or adjust as needed
        except (socket.error, BrokenPipeError):
            print("Client disconnected.")
            break

def server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 65432))
    server_socket.listen()
    print("Server is listening for connections...")

    while True:
        conn, addr = server_socket.accept()
        print('Connected by', addr)
        client_thread = threading.Thread(target=handle_client, args=(conn,))
        client_thread.start()

    server_socket.close()

# Start emotion detection in a separate thread
emotion_thread = threading.Thread(target=detect_emotions)
emotion_thread.start()

# Start the server in the main thread
server()

C:\Users\Yash\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Yash\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Server is listening for connections...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 572ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━

In [ ]:
import threading
import socket
import time
import cv2
import numpy as np
from collections import Counter
from keras.models import load_model
from keras.preprocessing.image import img_to_array

# Load the face detector and emotion recognition model
face_classifier = cv2.CascadeClassifier(r'C://emotion//haarcascade_frontalface_default (1).xml')
classifier = load_model(r'C://emotion//model.h5')

# Define emotion labels and initialize global variables
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
emotions_list = []
latest_emotion = "Neutral"
current_emotion = "Neutral"

def calculate_mode(emotions):
    """Calculate the most frequent emotion from the list."""
    filtered_emotions = [emotion for emotion in emotions if emotion != 'Neutral']
    if filtered_emotions:
        emotion_counts = Counter(filtered_emotions)
        most_common_emotion = emotion_counts.most_common(1)[0][0]
        return most_common_emotion
    return "Neutral"

def detect_emotions():
    """Capture video, detect faces, predict emotions, and update the latest detected emotion."""
    global latest_emotion
    global current_emotion
    global emotions_list

    cap = cv2.VideoCapture(0)

    while True:
        _, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray)

        detected_emotions = []

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)
            roi_gray = gray[y:y + h, x:x + w]
            roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

            if np.sum([roi_gray]) != 0:
                roi = roi_gray.astype('float') / 255.0
                roi = img_to_array(roi)
                roi = np.expand_dims(roi, axis=0)

                prediction = classifier.predict(roi)[0]
                label = emotion_labels[prediction.argmax()]
                current_emotion = label
                detected_emotions.append(label)
            else:
                cv2.putText(frame, 'No Faces', (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Update the global emotions list and calculate mode
        if detected_emotions:
            emotions_list.extend(detected_emotions)
        latest_emotion = calculate_mode(emotions_list)

        # Display current and most frequent emotion
        if current_emotion:
            cv2.putText(frame, f'Current Emotion: {current_emotion}', (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        if latest_emotion:
            cv2.putText(frame, f'Most Frequent Emotion: {latest_emotion}', (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow('Emotion Detector', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def handle_client(conn):
    """Handle communication with the client."""
    global latest_emotion
    global current_emotion
    while True:
        try:
            if latest_emotion and current_emotion:
                # Send both current and most frequent emotions
                message = f"Current Emotion: {current_emotion}\nMost Frequent Emotion: {latest_emotion}"
                conn.sendall(message.encode())
            time.sleep(1)  # Update every second or adjust as needed
        except (socket.error, BrokenPipeError):
            print("Client disconnected.")
            break

def server():
    """Set up the server to listen for connections and handle clients."""
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 65432))
    server_socket.listen()
    print("Server is listening for connections...")

    while True:
        conn, addr = server_socket.accept()
        print('Connected by', addr)
        client_thread = threading.Thread(target=handle_client, args=(conn,))
        client_thread.start()

    server_socket.close()

# Start emotion detection in a separate thread
emotion_thread = threading.Thread(target=detect_emotions)
emotion_thread.start()

# Start the server in the main thread
server()



C:\Users\Yash\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Yash\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Server is listening for connections...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━